In [33]:
#Import libraries
import pandas as pd
from pomegranate import *

In [40]:
#Read data
df = pd.read_excel('India_Test_Stats.xlsx', sheet_name='Sheet1')
#Dropping not required columns
df.drop(['Margin','Opposition','Start Date'],axis=1, inplace = True)

In [41]:
df.head()

,Ashwin,Result,Toss,Bat,Location
0,Y,won,lost,2nd,Home
1,Y,won,won,1st,Home
2,Y,draw,lost,2nd,Home
3,Y,lost,lost,2nd,Away
4,Y,lost,won,1st,Away


In [42]:
def CalculatePriorProbability(df, column):
    priorProbDict = {}
    type(priorProbDict)
    columns = list(df)
    
    for j in df[column].unique():
        priorProbDict[j] = df[df[column] == j].count().iloc[0] / df[[column]].count().iloc[0]
        priorProbDict[j] = df[df[column] == j].count().iloc[0] / df[[column]].count().iloc[0]
    return priorProbDict

In [43]:
dict = CalculatePriorProbability(df, 'Ashwin')
dict

{'Y': 0.8235294117647058, 'N': 0.17647058823529413}

In [44]:
def CalculateConditionalProbability(df, Location, AshwinPlaying):
    condProbDict = []
   
    columns = list(df)
    
    priorProb = CalculatePriorProbability(df, Location)
    for j in df[Location].unique():
        for k in df[AshwinPlaying].unique():
            val = df[(df[Location] == j) & (df[AshwinPlaying]  == k)].count().iloc[0] / df[[Location]].count().iloc[0]/priorProb[j]
            
            condProbDict.append([j,k,val])
    return condProbDict


In [45]:
#Data Description
arr = CalculateConditionalProbability(df, 'Location', 'Ashwin')
arr

[['Home', 'Y', 1.0],
 ['Home', 'N', 0.0],
 ['Away', 'Y', 0.6428571428571428],
 ['Away', 'N', 0.35714285714285715]]

In [22]:
arr = CalculateConditionalProbability(df, 'Toss', 'Bat')
arr

[['lost', '2nd', 0.7777777777777777],
 ['lost', '1st', 0.2222222222222222],
 ['won', '2nd', 0.1],
 ['won', '1st', 0.9]]

In [46]:
arr = CalculateConditionalProbability(df, 'Bat', 'Result')
arr

[['2nd', 'won', 0.41025641025641024],
 ['2nd', 'draw', 0.25641025641025644],
 ['2nd', 'lost', 0.33333333333333337],
 ['1st', 'won', 0.6739130434782609],
 ['1st', 'draw', 0.13043478260869565],
 ['1st', 'lost', 0.1956521739130435]]

In [47]:
#Construction of Bayesian Network goes here 
location = DiscreteDistribution(CalculatePriorProbability(df, 'Location'))
toss = DiscreteDistribution(CalculatePriorProbability(df, 'Toss'))

ashwin = ConditionalProbabilityTable(CalculateConditionalProbability(df, 'Location', 'Ashwin'),[location])
batting = ConditionalProbabilityTable(CalculateConditionalProbability(df, 'Toss', 'Bat'), [toss])
result = ConditionalProbabilityTable(CalculateConditionalProbability(df, 'Bat', 'Result'), [batting])

In [48]:
# State objects hold both the distribution, and a high level name.
sLocation = State(location, name="Location")
sToss = State(toss, name="Toss")
sBatting = State(batting, name="Batting")
sAshwin = State(ashwin, name="Ashwin")
sResult = State(result, name="Result")

In [49]:

# Create the Bayesian network object with a useful name
model = BayesianNetwork("Ashwin Playing Problem")

# Add the three states to the network 
model.add_states(sLocation, sToss, sBatting, sAshwin, sResult)

In [50]:
model.add_edge(sLocation, sAshwin)
model.add_edge(sToss, sBatting)
model.add_edge(sBatting, sResult)

In [51]:
model.bake()

In [54]:
#Solution for part 4
# a) #India winning, batting 2nd, Ashwin playing
model.probability([None, None, None,'2nd', 'Y', 'won'])

1.0

In [55]:
# b) India winning, batting 2nd, Ashwin not playing
model.probability([None, None, None,'2nd', 'N', 'won'])

1.0

In [56]:
# c) India losing, batting 2nd, Ashwin playing
model.probability([None, None, None,'2nd', 'Y', 'lost'])

1.0

In [57]:
# d) India losing, batting 2nd, Ashwin not playing
model.probability([None, None, None,'2nd', 'N', 'lost'])

1.0